以下の設定になっていること  

ランタイム > ランタイムのタイプを変更  
ハードウェアアクセラレータ：GPU  
ランタイムの仕様：ハイメモリ  

編集 > ノートブックの設定  
ハードウェアアクセラレータ：GPU  
ランタイムの仕様：ハイメモリ  

In [1]:
# googleドライブをマウント
from google.colab import drive 
drive.mount('/content/drive')
# 表示されるリンクをクリックして、アクセスを許可して、最後に表示される文字列を以下の入力欄に入れる

Mounted at /content/drive


In [2]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [3]:
# importで使う必要があるので、インストールがランタイム切れるごとに必要
# インストール後にランタイムの再起動を行わないとT5Tokenizerが見つからない
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# ドライブに保存してるものでインストール
!pip install -e transformers

# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

Obtaining file:///content/drive/My%20Drive/work/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 6.3 MB/s 
     |████████████████████████████████| 3.3 MB 53.3 MB/s 
  Running setup.py develop for transformers
     |████████████████████████████████| 159 kB 9.6 MB/s 
     |████████████████████████████████| 243 kB 65.6 MB/s 
     |████████████████████████████████| 69 kB 9.6 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0
     |████████████████████████████████| 1.1 MB 8.2 MB/s 


In [1]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


CLM（Causal Language Modeling）: GPT、GPT-2

・model_name_or_path: モデルのチェックポイント（モデルを最初から学習しない場合）  
・model_type: モデルの種別（モデルを最初から学習する場合）  
・config_name: コンフィグ名（model_nameと同じでない場合）  
・tokenizer_name: トークナイザー名（model_nameと同じでない場合）  
・cache_dir: キャッシュフォルダ  
・use_fast_tokenizer: Fastトークナイザーを使用するかどうか  
・model_revision: 使用するモデルの特定のバージョン  
・use_auth_token: 「transformers-cli login」の実行時に生成されたトークンを使用するかどうか  

・dataset_name: データセット名  
・dataset_config_name: データセットのコンフィグ名  
・train_file: 学習データ（テキストファイル）  
・validation_file: 検証データ（テキストファイル）  
・overwrite_cache: キャッシュの上書き  
・validation_split_percentage: 学習データから使われる検証データの割合（検証データがない場合）  
・max_seq_length: トークン化後の最大合計入力シーケンス長  
・preprocessing_num_workers: 前処理に使用するプロセス数  
・block_size: トークン化後のオプションの入力シーケンス長  
・max_train_samples: 学習データの最大数  
・max_val_samples: 検証データの最大数  

    # GPT2のモデルファイルを指定
    --model_name_or_path=rinna/japanese-gpt2-medium \
    # 学習ファイル
    --train_file=train.txt \
    # 評価データファイル
    --validation_file=train.txt \
    # トレーニングを実施する
    --do_train \
    # 評価を実施する
    --do_eval \
    # 学習回数（エポック数）
    --num_train_epochs=30 \
    # チェックポイントの保存間隔
    --save_steps=5000 \
    # チェックポイントの保持数
    --save_total_limit=3 \
    # T5Tokenizer.model_max_length=1024をチャンクサイズとして使用するかblock_sizeで指定するかを設定する（設定しないとtokenizerの超大なサイズから1024になる）メモリに乗るように調整する必要がある（バッチサイズとの兼ね合い）
    --block_size=512 \
    # GPU1つあたりの学習バッチサイズ
    --per_device_train_batch_size=2 \
    # GPU1つあたりの評価バッチサイズ
    --per_device_eval_batch_size=2 \
    # モデルとチェックポイントの出力先
    --output_dir=output/ \
    # 出力先の上書きの許可
    --overwrite_output_dir=True \
    # T5Tokenizerで高速化ライブラリがあれば使用する
    --use_fast_tokenizer=False

### 学習する回数
エポック数 * (データ文字数 / ブロックサイズ):1つの学習データ / バッチサイズ  
    エポック数 = 1  
    データ文字数 = 22142  
    データの文字数だと数字が合わないので、tokenizeした結果の形態素数による？
    ブロックサイズ = 512  
    データ文字数 / ブロックサイズ = 43  
    バッチサイズ = 1  
    Total optimization steps = 29  


バッチサイズ = 2  
バッチサイズを2にすることで、トータルの実行回数が減っている  
Total optimization steps = 15  

In [2]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train_all_ent.txt \
    --do_train \
    --num_train_epochs=500 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --block_size=256 \
    --per_device_train_batch_size=4 \
    --output_dir=output_all/ \
    --overwrite_output_dir=True \
    --use_fast_tokenizer=False

09/07/2021 11:46:25 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
09/07/2021 11:46:25 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=output_all/, overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=500.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Sep07_11-46-25_90d824060d5a, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, logging_steps=500, save_strategy=IntervalStrategy.STEPS, save_steps=5000, save_total_limit=3, no_cuda=False, seed=42, fp16=False, fp16_opt_leve

In [ ]:
# from transformers import T5Tokenizer, AutoModelForCausalLM

# # トークナイザーとモデルの準備
# tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
# model = AutoModelForCausalLM.from_pretrained("output/")

In [3]:
# もちろんだが、Autoでも直指定でも同じ結果にはなっている
# https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel
from transformers import T5Tokenizer, GPT2LMHeadModel

# GPU判定
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("output_all/")
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [4]:
# 推論
# https://huggingface.co/blog/how-to-generate
# https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate
input = tokenizer.encode("左のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブ これがデビュー戦緊張はしているがどんどん前に出て行きたい
左のジャブ これがここから突破口を音
左のジャブ これがデビュー戦。
左のジャブ これがデビュー戦。
左のジャブ これがデビュー戦。
左のジャブ これがここから突破口を音
左のジャブ これがデビュー戦。
左のジャブ これがデビュー戦。
左のジャブ これがデビュー戦。
左のジャブ これがデビュー戦緊張はしているがどんどん前に出て行きたい


In [5]:
input = tokenizer.encode("左のフック", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のフック これがデビュー戦。
左のフック ここはボディに打ち込んでいきます。
左のフック ここから大金星あげてくるのか
左のフック これがデビュー戦。
左のフック ここはボディに打ち込んでいます
左のフック アッパーからersonのパンチがnimal_artを捉えるosition_ocation。
左のフック 第2ラウンドはersonがダウンを覚えました
左のフック アッパーからersonのパンチへと変わっていきます
左のフック ここはちょっと距離を詰めていこうというこのerson。
左のフック ここはちょっと硬いですね


In [6]:
input = tokenizer.encode("ボディ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた
ボディ 右の誰かぐらっときた


In [7]:
input = tokenizer.encode("右のストレート", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のストレート これがデビュー戦。
右のストレート ここはガード固めて。
右のストレート これがデビュー戦。
右のストレート これがデビュー戦。
右のストレート ここはちょっと気味で右が入りました。
右のストレート これがデビュー戦。
右のストレート これがデビュー戦。
右のストレート これがデビュー戦。
右のストレート これがデビュー戦緊張はしているがどんどん前に出て行きたい
右のストレート これがデビュー戦。


In [8]:
input = tokenizer.encode("ガードの上から", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上から eaponを打っていくのは守りerson。
ガードの上から こちらもデビュー戦portで一花咲かせるか藤川遥29歳。
ガードの上から どんどん落ち込んでいている
ガードの上から eaponを打って行きます
ガードの上から どんどん落ち込んでいている
ガードの上から eaponを打っていくのは守りerson。
ガードの上から 森青葉。
ガードの上から eaponを打って行きますが。
ガードの上から どんどん落ち込んでいている
ガードの上から eaponを打っていくのは平井大


tokenizerの中身を確認  
\<s\>の意味合いを表示  


In [ ]:
# model.generateの結果はtokenizerのindexベクトル
output[4]

tensor([    9,  5682,    10, 12276,     2,     9,     0,    20,  2115,    18,
         5456,  2199,     7,    80,     0,    10,   819,     8,     2,     2,
            2,     2,     2])

In [ ]:
# 記号の意味
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
# 記号に対応するindex
tokenizer.all_special_ids

[1, 2, 0, 5, 3, 4, 6]

{"bos_token": "\<s\>", "eos_token": "\</s\>", "unk_token": "<unk>", "sep_token": "[SEP]", "pad_token": "[PAD]", "cls_token": "[CLS]", "mask_token": "[MASK]"}  
bos_token: 文の先頭（Begin of sequence token）  
eos_token: 文のおしり（End of Sequence token）  
unk_token: IDに変換できない文字（Unknown token）  
sep_token: 文と文を区切り目（The separator token）  
pad_token: パッディング（The token used for padding）  
cls_token: 分類用（cls_token）  
mask_token: マスク（The token used for masking values）  

* https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.batch_decode  
sequences : torch.Tensorの配列を入力値として指定  
  トークン化された入力IDのリスト  
skip_special_tokens : デコード時に特殊なトークンを削除するかどうか(eos_tokenとかを消す)(デフォルト:False)  
clean_up_tokenization_spaces : トークン化スペースをクリーンアップするかどうか(デフォルト:True)  

* https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.encode  

text (str, List[str] or List[int]) – 入力文字列  

text_pair (str, List[str] or List[int], optional) – ペアとなるもう一つを入力する場合のオプション  

add_special_tokens (bool, optional, defaults to True) – 上記で定義していない特別なトークンをモデルに適用するか

padding (bool, str or PaddingStrategy, optional, defaults to False) –パディングして入力シーケンスを揃える場合  

truncation (bool, str or TruncationStrategy, optional, defaults to False) –逆に長過ぎる場合に、一定の長さに揃える場合

max_length (int, optional) –トランケーション・パディングで使用するオプション

stride (int, optional, defaults to 0) – max_lengthで切り捨てられたのを調整する  

is_split_into_words (bool, optional, defaults to False) – 単語分割が既にされている場合True

pad_to_multiple_of (int, optional) – 指定された値の倍数になるようにシーケンスをパッドする  

return_tensors (str or TensorType, optional) – python整数のリストの代わりにテンソルを返す  
'tf': Return TensorFlow tf.constant objects.  
'pt': Return PyTorch torch.Tensor objects.  
'np': Return Numpy np.ndarray objects.  

* https://huggingface.co/blog/how-to-generate  
* https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate  
* https://note.com/npaka/n/n5d296d8ae26d  
* https://note.com/npaka/n/n96dde45fdf8d  

### GPT2LMHeadModel.generateのオプションを確認  

input_ids (torch.LongTensor of shape (batch_size, sequence_length), optional) – 入力シーケンス  

max_length (int, optional, defaults to model.config.max_length) – 生成されるシーケンスの最大長を指定（学習に使用した256の長さが良さそう）

max_new_tokens (int, optional, defaults to None) – 現在のトークン数に関係なく、生成されるシーケンスの最大長を指定

min_length (int, optional, defaults to 10) – 生成されるシーケンスの最小の長さ

do_sample (bool, optional, defaults to False) – 単語予測にサンプリングを入れてランダム性を導入する。（デフォルトは greedy decoding の生成）  
https://zenn.dev/hellorusk/articles/1c0bef15057b1d  

early_stopping (bool, optional, defaults to False) – ビーム探索で、num_beams個の文が生成された時点で、ビーム探索を終了するかどうか  

num_beams (int, optional, defaults to 1) – ビームサーチを行うビームの数。1はビームサーチを行わないことを意味します。  

temperature (float, optional, defaults to 1.0) – 次のトークンの確率をモジュール化するために使用される値です。温度（デフォルト1、推奨0.7〜1.0）ボルツマン分布のパラメータ。小さい値ではランダムな補完が減り，0では決まりきった繰り返しの文になる。大きい値ではより様々な補完がされる。  

top_k (int, optional, defaults to 50) – top-k-filteringのために保持する最高確率の語彙トークンの数です。確率が大きめな候補からサンプリングしてランダム性を導入する際の候補を何個にするか。40が一般的に良い値  

top_p (float, optional, defaults to 1.0) – 生成テキストを累積確率に制限 (0で制限なし) float < 1に設定すると、top_p以上の確率を持つ最も確率の高いトークンのみが生成のために保持されます。  
https://zenn.dev/hellorusk/articles/1c0bef15057b1d#top-p-(nucleus)-sampling  

repetition_penalty (float, optional, defaults to 1.0) – 反復ペナルティのパラメータです。1.0はペナルティなし。すでに生成された単語や文脈に属する単語にペナルティを与えるために使用することができます。反復防止にはかなり効果的ですが、異なるモデルやユースケースには非常に敏感なようで、議論がある。  

pad_token_id (int, optional) – PADトークンを指定

bos_token_id (int, optional) – bosトークンを指定

eos_token_id (int, optional) – eosトークンを指定

length_penalty (float, optional, defaults to 1.0) – 長さに対する指数関数的なペナルティ。1.0はペナルティがないことを意味します。1.0未満の値を設定すると、モデルは短い配列を生成するようになり、1.0以上の値を設定すると、モデルは長い配列を生成するようになります。

no_repeat_ngram_size (int, optional, defaults to 0) – int > 0に設定すると、そのサイズのngramはすべて一度しか発生しません。最も一般的な n-grams ペナルティは、すでに見た n-gramsを作る可能性のある次の単語の確率を 0 に手動で設定することで、n-gramsが 2 回出現しないようにするものです。  

encoder_no_repeat_ngram_size (int, optional, defaults to 0) – int > 0に設定すると、encoder_input_idsに出現したそのサイズのすべてのngramは、decoder_input_idsには出現しません。

bad_words_ids (List[List[int]], optional) – 生成してはいけないトークンのidのリスト。トークンのIDは以下で確認  
tokenizer(bad_word, add_prefix_space=True).input_ids  

num_return_sequences (int, optional, defaults to 1) – バッチ内の各要素について、独立して計算された戻り値の配列の数(返却される結果の数)。返されるべき最高得点のBeamの数を設定します。ただし、num_return_sequences <= num_beams とします。

max_time (float, optional, defaults to None) – 計算の実行を許可する最大時間を秒単位で指定します。割り当てられた時間が経過しても、生成は現在のパスを終了します。  

attention_mask (torch.LongTensor of shape (batch_size, sequence_length), optional) – パディングされたトークンのインデックスに対してアテンションを行わないようにするためのマスクです。マスクの値は [0, 1] で、マスクされていないトークンには 1、マスクされたトークンには 0 です。提供されていない場合は、パッドトークンをマスクするinput_idsと同じ形のテンソルがデフォルトになります。attentionで予測するための配列を作るので、マスクすると候補に出なくなる。入力シーケンスに対して同じ長さで指定する

decoder_start_token_id (int, optional) – エンコーダ・デコーダモデルがbosとは異なるトークンでデコードを開始した場合、そのトークンのid。

use_cache – (bool, optional, defaults to True): 過去の最後のキー／バリューの注目度（モデルに該当する場合）を利用して、デコーディングを高速化するかどうか。  

num_beam_groups (int, optional, defaults to 1) – num_beamsを分割するグループの数（ビームの異なるグループ間の多様性を確保するため）。

diversity_penalty (float, optional, defaults to 0.0) – この値は、ある時点で他のグループのビームと同じトークンを生成した場合、ビームのスコアから差し引かれます。なお、ダイバーシティペナルティは、グループビーム検索が有効な場合にのみ有効です。  

prefix_allowed_tokens_fn – (Callable[[int, torch.Tensor], List[int]], optional):提供された場合、この関数は、各ステップで許可されたトークンのみにビーム検索を制約します。提供されない場合、制約は適用されません。この関数は2つの引数をとります：バッチID batch_id と input_id です。これは、バッチID batch_idと以前に生成されたトークンinput_idsを条件として、次の生成ステップで許可されたトークンのリストを返さなければなりません。この引数は、「自己回帰的実体検索」で説明されているように、接頭辞を条件とした制約付き生成に役立ちます。  

output_attentions (bool, optional, defaults to False) – すべてのアテンションレイヤーのアテンションテンソルを返すかどうか。  

output_hidden_states (bool, optional, defaults to False) – すべてのレイヤーの隠れた状態を返すかどうか。  

output_scores (bool, optional, defaults to False) – 予測スコアを返すかどうか。  

return_dict_in_generate (bool, optional, defaults to False) – 単なるタプルではなく、ModelOutputを返すかどうか。  

forced_bos_token_id (int, optional) – decoder_start_token_idの後に、最初に生成されるトークンとして強制的に使用するトークンのidです。mBARTのような多言語モデルで、最初に生成されるトークンがターゲット言語のトークンである必要がある場合に便利です。（一番最初に生成される単語を指定してしまう。）  

forced_eos_token_id (int, optional) – max_lengthに達したときに、最後に生成されたトークンとして強制的に使用するトークンのidです。(最後をわかりやすくして、途中で切られたのを知らせる)  

remove_invalid_values (bool, optional) – 生成方法がクラッシュするのを防ぐために、モデルの可能性のあるnanとinfの出力を削除するかどうか。remove_invalid_valuesを使うと生成が遅くなることに注意してください。  

synced_gpus (bool, optional, defaults to False) – max_lengthまでwhileループを続けて実行するかどうか  

最新の研究により、単純な Beam Search や Greedy Search が同じ単語列の繰り返しを発生させてしまうのは、decoding に問題があるのではなくモデルの学習自体に問題があるとされています。また、Top-K や Top-p のようなサンプリングによる decoding であってもそうした単語列の繰り返しは発生しうるそうです。  

In [9]:
input = tokenizer.encode("左のフック", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のフック</s> ここはちょっと<unk> 気味で右が入りました。 この<unk> の右というのはちょっと速いですね。 俺を仕留めてていきたいとイメージでしょうか</s> </s> </s> </s> </s> </s>
左のフック</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンドは残り10秒です。 さあ距離を詰めてきた、この第3ラウンド。</s> </s>
左のフック</s> ここはボディに打ち込んでいきます。 ボディに打ち込んでいく<unk> ersonです。 左のジャブあたりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> ここはちょっと<unk> 気味で右が入りました。 この<unk> の右というのはちょっと速いですね。ちょっと早いですね。 俺を仕留めてていきたいとイメージでしょうか</s>
左のフック</s> ここはちょっと<unk> 気味で右が入りました。 第2ラウンドは少し静かな戦いになりましたが。 第2ラウンド終了です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> ここはボディに打ち込んでいきます。 ボディに打ち込んでいきます。 ストレートネック ただここはバックステップを使って未来はClothing避けていきます</s> </s> </s> </s> </s>
左のフック</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終了です。 ここまで激しい打ち合いになっている両者の戦い。</s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終了です。 ここまで激しい打ち合いになっている両者の戦い。</s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> ここはちょっと<unk> 気味で右が入りました。 この<unk> の右というのはちょっと速いですね。 ちょっと早いですね。</s> </s> </s> </s> </s> </s> </s> </s>

In [10]:
input = tokenizer.encode("ラッシュラッシュ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ラッシュラッシュ</s> これありますからね。 ガード上げて距離を詰めていく<unk> erson。 右のジャブ。 そっからこの次の展開を考えているこの<unk> octrine_<unk> ethod_<unk> therなんですがどうですか、これ距離感というの<unk> itle_<unk> therどう思いますか?</s>
ラッシュラッシュ</s> すごいですね。 はい。 ここはジャブを打っていって、残り2ラウンド。 ここは最終ラウンドです。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> すごいですね。 はい。 ここは右のジャブ。 このジャブを打っていって、最後のラウンドに持って行きました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> ここはジャブを打っていけばきた子は<unk> _<unk> umberでも食った気しないとは右の<unk> eaponと左の服が溶けて連絡あったうちらになっている第5ラウンド最後打ち合いになりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> これありますからね。 ガード上げて距離を詰めていく<unk> erson。 右のジャブ。 これやはり<unk> ersonの方が尻上がりに来る感じがしますね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>

In [11]:
input = tokenizer.encode("ダウン", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ダウン</s> <unk> ersonです。 ここは右のフック。 ダウンですね。 こういうの長い左のジャブを伸ばしていく<unk> erson。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ダウン</s> そこまでは<unk> osition_<unk> ocationがどんどん中に入っていきましたが。。。 そこの表情ですよ。 これもうこれなんか狙ってる感じの<unk> nimal_<unk> artしますよね</s>
ダウン</s> ここは美味しいですね。 いろいろ残りも入れていきました。 本人もそれを意識しだしてるんじゃないすかね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ダウン</s> そこまでは圧倒していたんですが最後の。。。 ここはバックステップ少しダウンを奪いましたが。。。 ただここはバックステップも残り30秒。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ダウン</s> <unk> nimal_<unk> artがこれは試合をひっくり返したと言ってもいいでしょうかあのまま走り流れ込んでいたらあるいは<unk> ersonだったかもしれませんが見事にその力でもってきました。</s> </s> </s> </s> </s>
ダウン</s> ここは美味しいですね。 いろいろ残りも入れていきました。 本人もそれを意識しだしてるんじゃないすかね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ダウン</s> <unk> nimal_<unk> artが曲がりましたが。。。。 聞いたと見ればパンチを集める力 そしてパンチを集める体が浮きました</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ダウン</s> <unk>

In [12]:
input = tokenizer.encode("あーとここで倒れた", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


あーとここで倒れた</s> かなり体力が限界かと思われたんですが<unk> erson声を出しながら前に出ます。 試合は序盤からどんどん行っている</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> ま復帰後はま復帰後初勝利およそ4年ぶりの勝利ということになりました 一方の<unk> ature_Colorコーナーの永遠ですが<unk> erson<unk> itle_<unk> ther</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 担架で運ばれて行きました。 まあしかし<unk> ersonにとっては前回が復帰後初勝利そして日本ランクも手にした一戦になりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 担架で運ばれて行きました。 これで当たればというところですが第5ラウンドが終了しましたのファイナルラウンド1ラウンドのみとなります</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> すでに第2ラウンドが終了しています。 まずは<unk> imeのパンチラ<unk> nimal_<unk> artを捉えるシーン。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> すでに第2ラウンドが終了しています。 まずは<unk> imeです。 まあ幼い<unk> erson<unk> osition_<unk> ocationですから甘いですね。</s> </s> </s> </s> </s> </s> </s> </s> </s

In [13]:
input = tokenizer.encode("ここで倒れた", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.85, temperature=0.7, min_length=64, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ここで倒れた</s> が、まだ大丈夫だとそういう機会にしたいと言っていましたからその寮に一礼をして。 まずは<unk> ature_Colorコーナーの<unk> ersonです。 まず入場してきたのが<unk> ature_Colorコーナー<unk> erson35歳です。 まず入場してきたのは<unk> ature_Colorコーナー<unk> erson35歳です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ここで倒れた</s> と思ったら今度は右の<unk> eaponが<unk> nimal_<unk> artをとらえました なかなかない中に入っていけるかどうか岩崎圭介は前回よりもすごく<unk> nimal_<unk> artを使ってない入らせない<unk> portしてますね そうですね あのやっぱりお互いにクリーンヒットがないんでここで栗拾い形になりましたね</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ここで倒れた</s> ということですが。 まみてる以上に対峙するとはいあれ思った以上に遠いなって物が溜まりにくいのかなと思いますね 実際、<unk> roduct_<unk> therの<unk> port見ててもそのスピードと手数の多さにちょっと真ん中が先週ママ対応しきれてないなーっていうのは思いましたほど</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ここで倒れた</s> ということですが。 まこの試合は1年4ヶ月ぶりの復帰戦のことになり

In [14]:
input = tokenizer.encode("ガードの上から連打", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上から連打</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 リング中央<unk> nimal_<unk> artの後ろには<unk> imeが。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上から連打</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 ガードの上からでもしっかり手数を出して。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上から連打</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 リング中央には秘密ですがそれは<unk> _<unk> therで一緒にしてClothingが飛んできました</s> </s> </s> </s> </s> </s> </s> </s>
ガードの上から連打</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 リング中央<unk> nimal_<unk> artの横にあるのは<unk> _<unk> ersonです</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上から連打</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 リング中央には秘密ですがそれは<unk> _<unk> therで一緒にしてClothingが飛んできました</s> </s> </s> </s> </s> </s> </s> </s>
ガードの上から連打</s> <unk> octrine_<unk> ethod_<unk> ther。。。 ボディに打ち込んでいます。 ストレートネック <unk> octrine_<unk> ethod_<unk> therの上からでもしっかり手数を出して。</s>
ガードの上から連打</s> <unk> octrine_<unk> ethod_<unk> ther。 リング中央

In [15]:
input = tokenizer.encode("倒れたノックアウトです。", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


倒れたノックアウトです。</s> ここまで2勝1敗1分け、神田桃子。 今リングインです。 <unk> choolでは<unk> osition_<unk> ocationを務めてきました</s> </s>
倒れたノックアウトです。</s> これで戦績は1勝6敗3分け。 いまだ価値がありません。 神田桃子34歳。 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れたノックアウトです。</s> これこの試合の明暗を分けました。 何それ。 何それ。 今この<unk> ersonもパンチを返して行きました</s> </s> </s> </s> </s> </s> </s>
倒れたノックアウトです。</s> ここまで4.4勝日本拳法で培ったと乗り鉄左のボディにも入れてきましたが堂々としますね</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れたノックアウトです。</s> これで終わりましたか今のですね やっぱりそうですねはいそして最後のダウンのシーンです これが本当かっぽい<unk> portしてますよね</s>
倒れたノックアウトです。</s> これで終了です判定は3体で落ち着いた試合展開を見せています<unk> です。 ここまでの試合手数では間違いなく<unk> ersonです</s> </s> </s> </s> </s> </s> </s>
倒れたノックアウトです。</s> これで終わりでしょうか。 これで戦績は1勝6敗2分け。 神田桃子34歳。 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れたノックアウトです。</s> これで終わりましたか今のです やっぱりそうですねはいそして藤井ステーション江藤会長もかなり<unk> nimal_<unk> artを使っています</s> </s> </s>
倒れたノックアウトです。</s> これで終わりましたか今のですね。 これ見てやっぱり<unk> erson<unk> itle_<unk> therの距離なんでしょうかね</s> </s> </s> </s> </s> </s> </s> </s>
倒れたノックアウトです。</s> ここまでは<u

In [16]:
input = tokenizer.encode("倒れたKOです。", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


倒れた<unk> です。</s> ここは右のストレート。 これ右の<unk> octrine_<unk> ethod_<unk> ther打ってきました <unk> ersonです</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> これ終了。 やっぱりもう10年ぐらいになる選手なので様々な経験をされていますしたりしたから追う者にとってはあの憧れる存在だと思います</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> ここは右のボディ。 やはり右のフックが当たっています。 倒れたまま、というのもありますよね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> そこに対して右のカウンターを狙っています。 相手の家終わりを狙っている強いていたミラクルハンマー。 その辺りがまあここから終盤にかけてポイントになってくるんですがどうでしょうね</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> ここは右のボディ。 やはり入ってきたところに右を合わせる力はこのカウンターパンチですねそういったところが一つか気になってくる。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> 試合は終盤に入ってきます。第5ラウンドです。序盤からどんどんパンチを打っているのは<unk> です。 ただ、<unk> もパンチを返していきます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>

In [17]:
input = tokenizer.encode("右のフックから左右連打", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のフックから左右連打</s> これがデビュー戦。 ボディに打ち込んでいます。 リングは中央です。 第2ラウンド終了です。 先ほどの神です</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> これがデビュー戦です。 パンチをもらっているシーンもありましたが、ただ冷静に試合展開を進めているのは<unk> ersonですかね。</s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> しかしここは両者。 パンチラ工作シーンです。 左の<unk> octrine_<unk> ethod_<unk> ther。</s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> ここはボディに打ち込んでいきます。 ボディに打ち込んでいく<unk> ersonです。 <unk> ersonはガード上げて中に入り込もうとするところ右のボディ。</s>
右のフックから左右連打</s> ここはボディに打ち込んでいきます。 ボディに打ち込んでいく<unk> ersonです。 <unk> ersonはガード上げて中に入り込もうとするところ右のボディ。</s>
右のフックから左右連打</s> おまけまあこれは来るわ<unk> nimal_<unk> art<unk> ature_Colorするよね答えありませんよね。</s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> すごいパンチが入りました。 第2ラウンド終了です。 ここまで激しい打ち合いになっている両者の戦い。 第2ラウンドです。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> 。 ボディに打ち込んでいます。 ストレートネック 第4ラウンド終了です。 ここまで激しい打ち合いになっている両者の戦い。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> これがデビュー戦。 第3ラウンドは落ち着いた試合展開。 右のストレートパンチを打ちながら距離の測り愛と

In [18]:
input = tokenizer.encode("ガードの上からボディー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上からボディー</s> <unk> ersonは打ち負けていません 第2ラウンド終了です。 この大量何度もお互い手数を出していきました。 岩崎圭介は<unk> ersonの動きに動じずに自分のペースを掴んでパンチを当てられるかそう試合前か立っていました</s>
ガードの上からボディー</s> <unk> ersonにしてみてもなかなか休めないさあ右の<unk> eaponを打ってきました 中に入っていけるかどうか<unk> ersonです</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> <unk> ersonです <unk> erson。 激しい打ち合いになっている。 右のジャブ。 ここはガードを固めています<unk> erson。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> <unk> ersonでも動きを止められない<unk> erson。 中に入っていけるかどうか<unk> erson。 パンチを返してきたら中に入ってくる<unk> ersonに対して右のカウンターを狙っています</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> <unk> ersonにしてきます。 中に入っていけるかどうか<unk> ersonです。 上下の打ち分け<unk> erson。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> <unk> ersonです <unk> erson。

In [19]:
input = tokenizer.encode("ジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ジャブ</s> この辺りは持ち味を十分というところです。ただ距離を取ってみると。。。 ジャブを打っていって、いきなり距離を詰めても次第に力の左のボディです。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> この辺りは持ち味を十分というところです。ただ距離を取ってみると。。。 <unk> ocation_<unk> therのフックを見せましたが大ちゃん残り10秒。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> この<unk> roduct_<unk> therどうでしょうか? それとかよりも今少し後ろに体重が移動しました。 左の<unk> roduct_<unk> ther!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> この<unk> roduct_<unk> therどうでしょうか? それとかよりも今少し後ろに体重が移動しました。 左の<unk> roduct_<unk> ther!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> この<unk> roduct_<unk> therどうでしょうか? それとかよりも今少し後ろに体重が移動しました。 左の<unk> roduct_<unk> ther!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> この<unk> roduct_<unk> therどうでしょうか? それとかよりも今少し後ろに体重が移動しました。 左の<unk> roduct_<unk> ther!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> この辺りは持ち味を十分というところです。 ただ距離を取ってみると。 第3ラウンドは残り10秒です。 張り込んでここで第3ラウンド終了です。</s>

In [20]:
input = tokenizer.encode("左のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブ</s> これがデビュー戦。 緊張はしているもののしっかり技術を持っているのは<unk> ersonです。 アマチュア戦績57.43勝14敗実績は十分にきゅうレビュー</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> これがデビュー戦。落ち着いた試合展開を見せている<unk> です。 序盤からパンチを打っているんですが、最後の。。。 左の<unk> octrine_<unk> ethod_<unk> ther。</s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> これがデビュー戦。 デビュー戦とは思えない落ち着き。落ち着いた試合展開を見せています<unk> です。 序盤からパンチを打っていく<unk> です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> これがデビュー戦。落ち着いた試合展開を見せている<unk> です。序盤からしっかり距離を取って、どんどんパンチを打っているのは<unk> です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> これがデビュー戦。落ち着いた試合展開を見せている<unk> です。 序盤からパンチを打っているんですが、最後の。。。 左の<unk> octrine_<unk> ethod_<unk> ther。</s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> これがデビュー戦。 <unk> rovinceのCity出身24歳浜口良性です。 <unk> choolボクシング部から真正ジムで今日デビュー戦を迎えました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ<

In [21]:
input = tokenizer.encode("右のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のジャブ</s> これがここから突破口を音 <unk> ersonです。 ただ距離を詰めていくのは<unk> ersonです。 左のジャブから右2で入れていく<unk> erson。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> これがここから突破口を音 <unk> ersonです。 ただ距離を詰めていくのは<unk> ersonです。 まずは<unk> ersonですが、左のジャブを打って行きました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> これがここから突破口を音 <unk> ersonです。 ただ距離を詰めるだけではなくて<unk> nimal_<unk> artはクールに行きたいんだけど話を完了していました</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> これがここから突破口を音 <unk> ersonです。 ただ距離を詰めるだけではなくて<unk> nimal_<unk> artはクールに行きたいんだけど話を完了していました</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> これが一つ<unk> ersonの形ではあります 距離を詰める<unk> ersonです <unk> ature_ColorClothingClothingそして<unk> ature_ColorClothingです</s> </s> </s> </s

In [22]:
input = tokenizer.encode("左のジャブからワンツー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブからワンツー</s> これがデビュー戦。 落ち着いた試合展開を見せている<unk> です。 パンチをもらっているシーンもありましたが、ただ冷静に試合展開を進めているのは<unk> ですかね。</s>
左のジャブからワンツー</s> これがデビュー戦。 落ち着いた試合展開を見せている<unk> です。 序盤からパンチを打っているんですが、最後の。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブからワンツー</s> この辺りのパンチの交差点の残り香の魅力ですね。 ここは右のボディから左お互い別の種類の集中力っていうのは本当にありませんね</s>
左のジャブからワンツー</s> この辺りのパンチの交差点の残り香の魅力ですね。 これですね。 さあ残り10秒です。</s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブからワンツー</s> これがデビュー戦。 落ち着いた試合展開を見せている<unk> です。 序盤からパンチを打っているんですが、最後の。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブからワンツー</s> この辺りのパンチの交差点の残り香の魅力ですね。 ここは右のボディから左<unk> ersonはこの左を狙ってますかあってますね</s> </s>
左のジャブからワンツー</s> これがデビュー戦。 落ち着いた試合展開を見せている<unk> です。 パンチをもらっているシーンもありましたが、ただ冷静に試合展開を進めているのは<unk> ですかね。</s>
左のジャブからワンツー</s> これがデビュー戦。 落ち着いた試合展開を見せている<unk> です。 序盤からパンチを打っているんですが、最後の。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブからワンツー</s> ここはちょっと<unk> 気味で右が入りました。 この<unk> の右というのはちょっと速いですね。 ちょっと早いですね</s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブからワンツー<

In [23]:
input = tokenizer.encode("左のアッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 第3ラウンドの後半。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 ここまで激しい打ち合いになっている両者の戦い。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 ちょっと様子見という第1ラウンドから第二ラウンド、様子見ながら拳を交えながら、というところがあります</s>
左のアッパー</s> ここはボディに打ち込んでいきます。 ガードを固めていますので状態を揺らしてパンチを<unk> していく<unk> 。 ただここもパンチを返していくのは<unk> ersonです</s> </s> </s> </s> </s> </s>
左のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 ちょっと様子見という第1ラウンドから第二ラウンド、様子見ながら拳を交えながら、というところがあります</s>
左のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終了です。 先ほどの神です この右のストレート中に入り込もうというところです。</s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 ちょっと様子見という第1ラウンドから第二ラウンド、様子見ながら拳を交えながら、というところがあります</s>
左のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 ちょっと様子見という第1ラウンドから第二ラウンド、様子見ながら拳を交えながら、というところがあります</s>
左のアッパー</s> 

In [24]:
input = tokenizer.encode("右のアッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のアッパー</s> ここはちょっと<unk> 気味で右が入りました。 この<unk> の右というのはちょっと速いですね。 俺を仕留めてていきたいとイメージでしょうか</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> ここはボディに打ち込んでいきます。 ボディに打ち込んでいく<unk> ersonです。 左のジャブから効果ボディを連打。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> ここはボディに打ち込んでいきます。 ボディに打ち込んでいく<unk> ersonです。 左のジャブから効果ボディを連打。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> ここはボディに打ち込んでいきます。 ガードを固めていますが。 ボディに打ち込んでいます。 ストレートネック 残り2秒。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 勝ったのは<unk> erson35歳です</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 ちょっと様子見という第1ラウンドから第二ラウンド、様子見ながら拳を交えながら、というところがあります</s>
右のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラウンド終わりました。 第3ラウンドの後半にかけて、両者のパンチが交錯するというシーンがありました。</s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> ここはボディに打ち込んでいきます。 リングは中央です。 第3ラ

In [25]:
input = tokenizer.encode("アッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


アッパー</s> 右の<unk> octrine_<unk> ethod_<unk> ther! 今リングイニです。 中村京太郎<unk> itle_<unk> ther</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> 右の<unk> octrine_<unk> ethod_<unk> ther! これも<unk> nimal_<unk> artをとらえた。 左のジャブから右の<unk> octrine_<unk> ethod_<unk> ther!</s>
アッパー</s> 右の<unk> octrine_<unk> ethod_<unk> ther! ここはガードを固めています<unk> です。 左のボディ!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> 右の<unk> octrine_<unk> ethod_<unk> ther! これも<unk> nimal_<unk> artをとらえた。 右のボディ!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> 右の<unk> octrine_<unk> ethod_<unk> ther! 今強いパンチが当たるそんなラウンドになりましたね。 第3ラウンドは少し静かな試合になりましたが。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> 右の<unk> octrine_<unk> ethod_<unk> ther! これも<unk> nimal_<unk> artをとらえた。 左のジャブから右の<unk> octrine_<unk> ethod_<unk> ther!</s>
アッパー</s> 右の<unk> octrine_<unk> ethod_<unk> ther! 今強いパンチが入りました。

In [26]:
input = tokenizer.encode("カウンター", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


カウンター</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 いいですねサウスポー。 第3ラウンド終わりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 いいシャブが出ています。 右の<unk> octrine_<unk> ethod_<unk> ther。</s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 ガードを固めながら右の<unk> octrine_<unk> ethod_<unk> ther。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 ガードを固めて距離を詰めていくのは守り<unk> erson。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 いいシャブが出ています。 右の<unk> octrine_<unk> ethod_<unk> ther。</s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> 右の息を切っています。 まこの試合に向けてますアマチュアでの実績は十分という<unk> 、ここまでは落ち着いた試合展開。 ただ序盤からどんどん手数を出して、自分の強みはスタミナだと語っていました</s>
カウンター</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 第6ラウンドは静かな戦いになっている依頼もそれとえらいね</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </

In [27]:
input = tokenizer.encode("試合展開", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


試合展開</s> 中盤にかけて手数を増やしてきた<unk> ersonです。 ただここもフットワークを生かして体が入れ替わります。 左の<unk> octrine_<unk> ethod_<unk> ther。</s>
試合展開</s> 中盤にかけて<unk> ersonが手数を出していく中、カウンターで未来の<unk> eaponを打ち返していく。 ただ、<unk> ersonもパンチを返していく。</s> </s> </s> </s>
試合展開</s> 中盤にかけて<unk> ersonがダウンを覚えましたが大丈夫でしょうかでのパンチ集めて行きました。 一方の設備ですが念願の初勝利に向けて第1ラウンドの立ち上げになります</s> </s>
試合展開</s> 右の<unk> octrine_<unk> ethod_<unk> ther! ここはガードを固めています<unk> です。 ただここは細かいパンチボディ店<unk> 。</s> </s> </s> </s>
試合展開</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 ここはガードを固めています<unk> です。 中に入っていきたい<unk> です。</s> </s> </s> </s> </s>
試合展開</s> 中盤にかけて<unk> ersonがダウンを覚えましたが大丈夫でしょうかでのパンチ集めて行きました。 試合は終盤に入ってきます</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開</s> 右の<unk> octrine_<unk> ethod_<unk> ther! これがデビュー戦緊張はしているがどんどん前に出て行きたい。 その通りの試合展開になっています</s> </s> </s> </s> </s> </s>
試合展開</s> 右の<unk> octrine_<unk> ethod_<unk> ther! ここはガードを固めています<unk> です。 中に入っていきたい<unk> です。</s> </s> </s> </s> </s>
試合展開</s> 右の<unk> octrine_<unk> ethod_<unk> ther。 ここはガードを固めています<unk> です。 中に入ってい

In [28]:
input = tokenizer.encode("試合展開について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


試合展開について</s> 右の<unk> octrine_<unk> ethod_<unk> ther! ここはしっかり中に入っていきたい。 ガードを固めながら中に入ろうとするのは<unk> ersonです</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> <unk> erson<unk> itle_<unk> therはこういう選手だったというのを今日の試合でやはりもう<unk> requency皆に分かってもらいたくないそういう気持ちで望んでいます</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 右の<unk> octrine_<unk> ethod_<unk> ther! ここはガードを固めています<unk> です。 まず中に入っていこうという<unk> です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 右の<unk> octrine_<unk> ethod_<unk> ther! ここは少し<unk> nimal_<unk> artが止まってきたか<unk> osition_<unk> ocation。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> もう<unk> requency<unk> erson<unk> itle_<unk> therが話していたのは<unk> osition_<unk> ocationが増しているのはある程度では<unk> ersonには負けてないぞどんどんこのスピードで押していけ</s>
試合展開について</s> 序盤の<unk> nimal_<unk> artのペイントプラスこの今のこのこの右からいきなり起こさないというのあったらもっとねあのいい展開にたかが<unk> osition_<unk> ocationにとって良い展開になっていますね</s> </s> </

In [29]:
input = tokenizer.encode("ボクシング", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボクシング</s> グローブがこれがデビュー戦という<unk> erson22歳<unk> rovinceのCity出身です。 2017年に<unk> requencyプロのライセンスを取得しましたが仕事や家庭の事情で試合ができなかった</s>
ボクシング</s> 35歳で6年ぶりの復帰戦とありましたがまぁ若いのある港にラウンド<unk> で敗れています<unk> erson<unk> itle_<unk> ther</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 35歳にして6年ぶりの復帰戦とありましたがまぁ若いのある港にラウンド<unk> で敗れています <unk> erson<unk> itle_<unk> ther</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 35歳にして6年ぶりの復帰戦とありましたがまぁ若いのある港にラウンド<unk> で敗れています <unk> erson<unk> itle_<unk> ther</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 35歳で6年ぶりの復帰戦とありましたがまぁ若いのある港にラウンド<unk> で敗れています <unk> erson<unk> itle_<unk> ther</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 引退してからの<unk> port人生。 今振り返れば。 あの<unk> がこんなに勝つとは思っていませんでしたと本人は話していました</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 35歳で6年ぶりの復帰戦とありましたがまぁ若いのある港にラウンド<unk> で敗れています <unk> erson<unk> itle_<unk> ther</s> </s> </

In [30]:
input = tokenizer.encode("ボクシングの試合について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボクシングの試合について</s> <unk> erson<unk> itle_<unk> therがいない<unk> osition_<unk> ocationの試合展開とは違った落ち着いた左のジャブを見せているなという印象ですが第4ラウンドが始まりました</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> そうですね ただ前に出ようとする少し明日から前に出ようとする<unk> ersonはどういった展開を見せるか残り10秒となって。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> そうですね 一年半ぶりの復帰戦ということになりますかね まあ その辺りのブランクについては<unk> erson試合前に話していました</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> そうですね この試合に向けてますかね ただ一つ不安なのが体だけじゃなくて気持ちの強さを見せたいと話していた中でやはり序盤の立ち上がり左のジャブが当たるのかどうかというところ。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> <unk> erson<unk> itle_<unk> therがすごくいいボクシングを見せましたね。 すごい接近戦がうまいなと思います。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> そうですね 前に出た方がパンチを集める力っていうのはすごくある <unk> osition_<unk

In [31]:
input = tokenizer.encode("今回のボクシングの試合について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


今回のボクシングの試合について</s> 自分の<unk> port人生の中でも<unk> ank良かったので今回は<unk> で勝ちたいと語っていました<unk> ersonです</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
今回のボクシングの試合について</s> 自分の<unk> port人生の中でも<unk> ank良かったので、まあこの時は買って何とかならんち無理をしないというところが自分の強みだと語っていたんですが、序盤からどんどん自分の<unk> port人生の中でも<unk> ank良かったなという試合になっています。</s> </s>
今回のボクシングの試合について</s> 自分の<unk> port人生は<unk> requency負けなしだから絶対に負けられないという話でいました。 この試合に向けてます</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
今回のボクシングの試合について</s> 自分の<unk> port人生は<unk> rovinceにかかっているんだと語っていた<unk> ersonですが序盤からどんどん自分のペースに持って行こうとする<unk> erson</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
今回のボクシングの試合について</s> 自分の<unk> port人生の中でも<unk> ank良かったので今回は<unk> で勝ちたいという話をしていました。 その辺りの<unk> erson<unk> itle_<unk> therの気持

In [32]:
input = tokenizer.encode("かなりのダメージを受けている", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


かなりのダメージを受けている</s> ここから後半戦に入っているところにつながった巻き返しを見せるんでしょうか。 ただ第4ラウンドもパンチを出しているのは<unk> ersonです。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
かなりのダメージを受けている</s> ここから再起戦となる。 キャリア49戦目、35歳の<unk>! 負けよりかそれほど高くないんですが場所学校1年生から空手キックボクシング小学校5年から<unk> portを始めました</s> </s> </s> </s>
かなりのダメージを受けている</s> ここから後半戦に入っているところにつながった巻き返しを見せるんでしょうか。 ただ第4ラウンドもパンチを出しているのは<unk> ersonです ただ第4ラウンドは<unk> ersonもパンチを返して行きました</s>
かなりのダメージを受けている</s> ここから後半戦に入っているところにつながった巻き返しを見せるんでしょうか。 そうですね巻き返していけるかどうかですね? だいぶダメージは効いていると思います。</s> </s> </s> </s> </s> </s> </s> </s>
かなりのダメージを受けている</s> ここから後半戦に入っているところにつながった巻き返しを見せるんでしょうか。 まずは<unk> ersonです。 ただこれまでの戦いはま復帰後の初勝利が絶対条件という風に語っていました</s> </s> </s> </s> </s>
かなりのダメージを受けている</s> ここから後半戦に入っているところにつながった巻き返しを見せるんでしょうか。 パンチをもらっているシーンもありましたがどんどん前に出て行くね</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
かなりのダメージを受けている</s> ここから後半戦に入っているところにつながった巻き返しを見せるんでしょうか。 パンチをもらっているシーンもありましたが、ただ冷静に試合展開を進めているのは<unk> ersonですかね。</s> </s> </s> </s> </s>
かなり